In [1]:
import pandas as pd

from tqdm.contrib.concurrent import thread_map

c:\Users\PC\anaconda3\envs\smart_menu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create dictionary

In [2]:
df = pd.read_excel('../data_sample/Data_Labeling.xlsx')

df.head()

,ImageName,VietnameseName,EnglishName,Price
0,001.jpeg,COMBO 1,COMBO 1,169000
1,001.jpeg,COMBO 2,COMBO 2,169000
2,001.jpeg,COMBO 3,COMBO 3,169000
3,001.jpeg,RƯỢU SOJU,SOJU,NOT GIVEN
4,001.jpeg,RƯỢU VODKA,VODKA,NOT GIVEN


In [3]:
df.tail()

,ImageName,VietnameseName,EnglishName,Price
15206,850.jpeg,SÒ ĐIỆP NƯỚNG,GRILLED SCALLOP,10000
15207,850.jpeg,KHÔ MỰC,DRIED SQUID,NOT GIVEN
15208,850.jpeg,CÁ SƠN NƯỚNG,GRILLED CARDINAL FISH,NOT GIVEN
15209,850.jpeg,CÁ CHỈ VÀNG,YELLOWSTRIPE SCAD,60000
15210,850.jpeg,CÁ LAO NƯỚNG,GRILLED RED CORNETFISH,99000


In [4]:
#get Vietnamese food names
food_names = df['VietnameseName'].values
type(food_names)

numpy.ndarray

In [5]:
#Clean text, remove punification number and etc
import re

def clean_food_name(food_name):
 
    return food_name.lower().strip()


In [8]:
food_vocabulary = thread_map(clean_food_name, food_names, max_workers=6)


100%|██████████| 15211/15211 [00:00<00:00, 338018.07it/s]


In [10]:

print(food_vocabulary[:20])


['combo 1', 'combo 2', 'combo 3', 'rượu soju', 'rượu vodka', 'tiger lon', 'tiger chai', 'tiger bạc chai', 'tiger bạc lon', 'coca', 'sprite', 'suối', 'bò húc', 'bia quy nhơn', 'bia bivina', 'strong bow', 'rau muống xào tỏi', 'mồng tơi xào tỏi', 'cải xào tỏi', 'đậu bắp xào tỏi']


In [11]:
from collections import Counter

corpus = Counter(food_vocabulary)
corpus.most_common(20)

[('khoai tây chiên', 67),
 ('nước suối', 57),
 ('trà đào', 52),
 ('chả ram tôm đất', 43),
 ('cơm chiên hải sản', 38),
 ('mì xào hải sản', 38),
 ('trà tắc', 33),
 ('trà chanh', 31),
 ('cháo hải sản', 30),
 ('pepsi', 30),
 ('bò húc', 28),
 ('cháo hàu', 28),
 ('gỏi hải sản', 27),
 ('mì xào bò', 27),
 ('cơm chiên trứng', 26),
 ('nước ngọt', 26),
 ('cơm chiên dương châu', 25),
 ('phô mai que', 25),
 ('sữa tươi trân châu đường đen', 24),
 ('coca', 23)]

In [12]:
#Save corpus
with open('food_vocabulary_raw.txt', 'w', encoding='utf-8') as f:
    for food, count in corpus.most_common():
        save_format = f"{food}${count}\n"
        f.write(save_format)




# Spell correct with symspell

In [13]:
import symspellpy
import random

import string

## Generate test cases

In [56]:
def random_delete(text, percent = 0.1):
    n = int(percent*len(text))+1
    text_aggumented = list(text)
    
    for i in range(n):
        k = random.randint(0, len(text_aggumented)-1)
        del text_aggumented[k]
        
    return "".join(text_aggumented)

def random_replace(text, percent = 0.2):
    n = int(percent*len(text))+1
    text_aggumented = list(text)
    
    for i in range(n):
        k = -1
        while k == -1 or text[k] == ' ':
            k = random.randint(0, len(text)-1)

        char = random.choice(string.ascii_uppercase)
        text_aggumented[k] = char
        
    return "".join(text_aggumented)

def random_swap(text, percent = 0.1):
    n = int(percent*len(text))+1
    text_aggumented = list(text)
    
    for i in range(n):
        k = -1
        while k == -1 or text[k] == ' ':
            k = random.randint(0, len(text)-1)
        h = k
        while h == k or text[h] == ' ':
            h = random.randint(0, len(text)-1)
        text_aggumented[k], text_aggumented[h] = text_aggumented[h], text_aggumented[k]
        
    return "".join(text_aggumented)

def lower(text):
    text = text.lower()
    return text

def random_text_aggument(text):
    k = random.randint(1, 2)
    random_agg = {0: random_delete, 1:random_replace, 2:random_swap}
    return random_agg[k](text, percent=0.2)
    
test = random_text_aggument('BELGIAN DARK 8.1%IBU 32 - 500ML (PHỞ ĐÙI + TRỨNG NON) thành pho')


In [60]:
TEST_CASE = random.choices(food_names, k = 500)
TEST_CASE = thread_map(lower, TEST_CASE, max_workers=6)
TEST_CASE_WRONG = thread_map(random_text_aggument, TEST_CASE, max_workers=6)

print(TEST_CASE[:7])
print(TEST_CASE_WRONG[:7])

100%|██████████| 500/500 [00:00<00:00, 249779.90it/s]

['cơm tấm sườn bì', 'chanh dây', 'da ếch rang muối sả', 'bánh tráng bơ ', 'trà lài golden (nhỏ)', 'sinh tố bơ dừa', 'kimuchi']
['cơờ nưm ấsmt bì', 'cahny dâh', 'da ếih rang msốc uả', 'bRnO tráng bơ ', 'RrZ làG OoPden (nhỏ)', 'sbnh tố iơ dừa', 'CimCchi']


## METRIC

In [15]:
def cer(pred, true):
    n = len(true)
    wrong = 0
    for c1, c2 in zip(pred, true):
        if c1 != c2:
            wrong += 1
    
    
    return (n - wrong)/n

def wer(pred, true):
    pred = pred.split()
    true = true.split()
    n = len(true)
    wrong = 0
    for c1, c2 in zip(pred, true):
        if c1 != c2:
            wrong += 1
    
    return (n - wrong)/n

print(cer("tomorrow now today and tomorrow", "tomorrow now today and tomoraow"))
wer("tomorrow now today and tomorrow", "tomorrow now today and tomoraow")


0.967741935483871


0.8

## Spell check

In [50]:
from symspellpy import SymSpell, Verbosity
from itertools import islice
import numpy as np

EDIT_DISTANCE = 7

spell_check = SymSpell(max_dictionary_edit_distance=7, prefix_length=10)
spell_check.load_dictionary('food_vocabulary_raw.txt', 0, 1, 
                                    encoding='utf-8', separator='$')


print(list(islice(spell_check.words.items(), 20)))



[('khoai tây chiên', 67), ('nước suối', 57), ('trà đào', 52), ('chả ram tôm đất', 43), ('cơm chiên hải sản', 38), ('mì xào hải sản', 38), ('trà tắc', 33), ('trà chanh', 31), ('cháo hải sản', 30), ('pepsi', 30), ('bò húc', 28), ('cháo hàu', 28), ('gỏi hải sản', 27), ('mì xào bò', 27), ('cơm chiên trứng', 26), ('nước ngọt', 26), ('cơm chiên dương châu', 25), ('phô mai que', 25), ('sữa tươi trân châu đường đen', 24), ('coca', 23)]


In [36]:
def correct_spell(text):
    suggestion = spell_check.lookup(text, Verbosity.ALL, include_unknown=True)
    return suggestion[0]._term

In [49]:
correct_spell('khoi tay chi')

'khoai tây chiên'

In [61]:
%%time
test_result = []
wer_result = []
cer_result = []
N = 7
for test_case, val in zip(TEST_CASE_WRONG, TEST_CASE):
    correct_text = correct_spell(test_case)
    
    test_result.append(correct_text)
        
        
    wer_result.append(
        wer(correct_text, val)
    )
    
    cer_result.append(
        cer(correct_text, val)
    )
    

print("Example: ")
print(TEST_CASE[:N])
print(TEST_CASE_WRONG[:N])
print(test_result[:N])

print("Metric:", f"WER: {np.mean(wer_result):.3f}", f"CER: {np.mean(cer_result):.3f}")



Example: 
['cơm tấm sườn bì', 'chanh dây', 'da ếch rang muối sả', 'bánh tráng bơ ', 'trà lài golden (nhỏ)', 'sinh tố bơ dừa', 'kimuchi']
['cơờ nưm ấsmt bì', 'cahny dâh', 'da ếih rang msốc uả', 'bRnO tráng bơ ', 'RrZ làG OoPden (nhỏ)', 'sbnh tố iơ dừa', 'CimCchi']
['cơm tấm sườn bì', 'chanh dây', 'da ếch rang muối sả', 'bánh tráng bơ', 'trà lài golden (nhỏ)', 'sinh tố bơ dừa', 'kim chi']
Metric: WER: 0.881 CER: 0.939
CPU times: total: 7.28 s
Wall time: 9.35 s


In [52]:
correct_spell("ca rong bien")

'canh rong biển'

In [53]:
correct_spell("go chiến maa")

'gà chiên mắm'